In [2]:
import pandas as pd
import os
import json
from collections import defaultdict
import re

In [35]:
# Pfad zum Ordner mit den JSON-Dateien
data_folder = 'data/instagram'

# Leere Liste zur Speicherung aller Kommentare
all_comments = []

# Durchlaufe alle Dateien im Ordner "data"
for filename in os.listdir(data_folder):
    if filename.endswith(".json"):
        filepath = os.path.join(data_folder, filename)
        
        # Vereinsname aus dem Dateinamen extrahieren (z.B. "verein1.json" -> "verein1")
        club_name = os.path.splitext(filename)[0].replace('_instacomments', '')

        # Öffne die JSON-Datei und lade den Inhalt
        with open(filepath, 'r', encoding='utf-8') as f:
            posts = json.load(f)

            # Falls das JSON eine Liste von Posts ist
            if isinstance(posts, list):
                # Extrahiere die Kommentare aus jedem Post
                for post in posts:
                    if 'comments' in post:
                        for comment in post['comments']:
                            all_comments.append({
                                'verein': club_name,  # Name des Vereins aus der Datei
                                'post': post.get('link', ''),  # Link zum Post (falls vorhanden)
                                'caption': post.get('caption', ''),  # Post-Caption (falls vorhanden)
                                'comment': comment,  # Kommentar-Text
                            })
            # Falls das JSON keine Liste ist (z.B. einzelne Posts)
            elif 'comments' in posts:
                # Extrahiere die Kommentare aus dem einzelnen Post
                for comment in posts['comments']:
                    all_comments.append({
                        'verein': club_name,  # Name des Vereins aus der Datei
                        'post': posts.get('link', ''),  # Link zum Post (falls vorhanden)
                        'caption': posts.get('caption', ''),  # Post-Caption (falls vorhanden)
                        'comment': comment,  # Kommentar-Text
                    })

# Speichern aller Kommentare in einem Pandas DataFrame
df = pd.DataFrame(all_comments)

df = df[~df['comment'].str.contains("Original-Audio", case=False, na=False)]
df = df[~df['comment'].str.contains("•", case=False, na=False)]

df = df.drop_duplicates(subset=['comment'])

df

,verein,post,caption,comment
0,staderennesfc,https://www.instagram.com/p/C8e7U9jAc5P/,"Brest puis Nantes, les affiches des derbies po...","On est 1 point devant les relégables, on perd ..."
1,staderennesfc,https://www.instagram.com/p/C8e7U9jAc5P/,"Brest puis Nantes, les affiches des derbies po...",Mm notre équipe amicale est meilleure que vous...
2,staderennesfc,https://www.instagram.com/p/C8e7U9jAc5P/,"Brest puis Nantes, les affiches des derbies po...",Ppl need to stop supporting this match fixing ...
3,staderennesfc,https://www.instagram.com/p/C8e7U9jAc5P/,"Brest puis Nantes, les affiches des derbies po...",Match fixedddd
4,staderennesfc,https://www.instagram.com/p/C8e7U9jAc5P/,"Brest puis Nantes, les affiches des derbies po...",Just investigate you will find lots of very co...
...,...,...,...,...
117884,fcchelsea,https://www.instagram.com/reel/DAePCASsGvm/,"Three points, four goals. 🗣️👊#CHEBHA #CFC #Che...",Innit star boy 🥶
117885,fcchelsea,https://www.instagram.com/reel/DAePCASsGvm/,"Three points, four goals. 🗣️👊#CHEBHA #CFC #Che...",🙌🙌🥶
117886,fcchelsea,https://www.instagram.com/reel/DAePCASsGvm/,"Three points, four goals. 🗣️👊#CHEBHA #CFC #Che...",🪄 💙
117889,fcchelsea,https://www.instagram.com/reel/DAePCASsGvm/,"Three points, four goals. 🗣️👊#CHEBHA #CFC #Che...",love you bro always💙🤍


In [36]:
posts_per_club = df.groupby('verein')['post'].nunique().reset_index()
posts_per_club.columns = ['Verein', 'Anzahl_Posts']

comments_per_club = df.groupby('verein')['comment'].count().reset_index()
comments_per_club.columns = ['Verein', 'Anzahl_Kommentare']

comments_per_club['avg_comments_per_post'] = comments_per_club['Anzahl_Kommentare'] / posts_per_club['Anzahl_Posts']


summary_df = pd.merge(posts_per_club, comments_per_club[['Verein', 'Anzahl_Kommentare', 'avg_comments_per_post']], on='Verein')

summary_df

,Verein,Anzahl_Posts,Anzahl_Kommentare,avg_comments_per_post
0,arsenal,239,3063,12.815900
1,avfc,156,2871,18.403846
2,bayer04,240,5863,24.429167
3,bvb,229,3292,14.375546
4,fcbayern,59,4895,82.966102
5,fcchelsea,19,10232,538.526316
6,fch,225,2301,10.226667
7,ipswich,107,1043,9.747664
8,lille,71,1729,24.352113
9,liverpoolfc,126,15148,120.222222


In [37]:
# Laden der Vereine und Spieler aus Ihrer JSON-Datei
with open('player.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [ ]:
# Liste zur Speicherung der Ergebnisse
results = []

# Sicherstellen, dass die 'comment'-Spalte keine NaN-Werte enthält
df['comment'] = df['comment'].fillna('')

# konvertieren der 'comment'-Spalte einmalig zu Kleinbuchstaben
df['comment_lower'] = df['comment'].str.lower()

# Verarbeitung jedes Vereins und seiner Spieler
for club in data['clubs']:
    club_name = club['club_name']
    # Filtern der Kommentare, die zu diesem Verein gehören
    df_club = df[df['verein'] == club_name].copy()

    df_club["comment"]
    
    # Überprüfen der Spalten in df_club
    print(f"Verarbeite Verein: {club_name}")
    print("Spalten in df_club:", df_club.columns.tolist())
    
    for player_name in club['players']:
        # Bereinigung des Spielernamens
        player_name = player_name.strip()
        
        # Aufteilen des Namens in Teile
        name_parts = player_name.split()
        # Erstellen von Varianten des Namens
        variants = set()
        variants.add(player_name.lower())  # Voller Name
        # Vorname(n)
        '''
        if len(name_parts) > 1:
            first_name = ' '.join(name_parts[:-1]).lower()
            variants.add(first_name)
        else:
            first_name = name_parts[0].lower()
            variants.add(first_name)
        '''
        # Nachname(n)
        last_name = name_parts[-1].lower()
        variants.add(last_name)
        print(variants)

        mask = df_club[df_club['comment'].str.contains('|'.join(variants), case=False, na=False)]
        # Extrahiere die Kommentare als Liste
        matching_comments = mask['comment'].tolist()
        
        '''
        # Erstellen des Regex-Musters
        pattern = r'\b(' + '|'.join(re.escape(v) for v in variants) + r')\b'
        regex = re.compile(pattern)
        
        # Anwenden des Regex auf die 'comment_lower'-Spalte
        mask = df_club['comment_lower'].apply(lambda x: bool(regex.search(x)))
        
        # Überprüfen der Spalten in df_club[mask]
        print(f"Verarbeite Spieler: {player_name}")
        print("Spalten in df_club[mask]:", df_club.loc[mask].columns.tolist())
        
        # Abrufen der passenden Kommentare mit .loc
        matching_comments = df_club.loc[mask, 'comment'].tolist()
        '''
        # Hinzufügen der Ergebnisse zur Liste
        results.append({
            'club': club_name,
            'player': player_name,
            'comments': matching_comments
        })
        
        
# Schreiben der Ergebnisse in eine neue JSON-Datei
with open('player_comments.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

Verarbeite Verein: vfb
Spalten in df_club: ['verein', 'post', 'caption', 'comment', 'comment_lower']
{'nübel', 'alexander nübel'}
{'dennis seimen', 'seimen'}
{'fabian bredlow', 'bredlow'}
{'stefan drljaca', 'drljaca'}
{'ameen al-dakhil', 'al-dakhil'}
{'jeff chabot', 'chabot'}
{'zagadou', 'dan-axel zagadou'}
{'rouault', 'anthony rouault'}
{'leonidas stergiou', 'stergiou'}
{'chase', 'anrie chase'}
{'hendriks', 'ramon hendriks'}
{'maximilian mittelstädt', 'mittelstädt'}
{'krätzig', 'frans krätzig'}
{'josha vagnoman', 'vagnoman'}
{'pascal stenzel', 'stenzel'}
{'angelo stiller', 'stiller'}
{'atakan karazor', 'karazor'}
{'keitel', 'yannik keitel'}
{'nartey', 'nikolas nartey'}
{'millot', 'enzo millot'}
{'fabian rieder', 'rieder'}
{'führich', 'chris führich'}
{'diehl', 'justin diehl'}
{'leweling', 'jamie leweling'}
{'raimund', 'luca raimund'}
{'woltemade', 'nick woltemade'}
{'ermedin demirovic', 'demirovic'}
{'undav', 'deniz undav'}
{'toure', 'el bilal toure'}
Verarbeite Verein: bayer04
Spalte

In [4]:
df = pd.read_csv("data/palmer_llm_test.csv")

In [5]:
spielername = 'Cole Palmer'
df_spieler = df[df['player'] == spielername]

In [22]:
def preprocess_json(players):
    # Leere Liste zur Speicherung aller Kommentare
    all_comments = []
    
    # Öffne die JSON-Datei und lade den Inhalt
    with open("data/instagram/vfb_instacomments.json", 'r', encoding='utf-8') as f:
        posts = json.load(f)

        # Falls das JSON eine Liste von Posts ist
        if isinstance(posts, list):
            # Extrahiere die Kommentare aus jedem Post
            for post in posts:
                if 'comments' in post:
                    for comment in post['comments']:
                        all_comments.append({
                            'post': post.get('link', ''),  # Link zum Post (falls vorhanden)
                            'caption': post.get('caption', ''),  # Post-Caption (falls vorhanden)
                            'comment': comment,  # Kommentar-Text
                        })
            # Falls das JSON keine Liste ist (z.B. einzelne Posts)
                elif 'comments' in posts:
                    # Extrahiere die Kommentare aus dem einzelnen Post
                    for comment in posts['comments']:
                        all_comments.append({
                            'post': posts.get('link', ''),  # Link zum Post (falls vorhanden)
                            'caption': posts.get('caption', ''),  # Post-Caption (falls vorhanden)
                            'comment': comment,  # Kommentar-Text
                        })

    # Speichern aller Kommentare in einem Pandas DataFrame
    df = pd.DataFrame(all_comments)
    df = df[~df['comment'].str.contains("Original-Audio", case=False, na=False)]
    df = df[~df['comment'].str.contains("•", case=False, na=False)]

    df = df.drop_duplicates(subset=['comment'])

    results = []

    for player_name in players:
        # Bereinigung des Spielernamens
        player_name = player_name.strip()
        
        # Aufteilen des Namens in Teile
        name_parts = player_name.split()
        # Erstellen von Varianten des Namens
        variants = set()
        variants.add(player_name.lower())  # Voller Name
    
        # Nachname(n)
        last_name = name_parts[-1].lower()
        variants.add(last_name)
        print(variants)

        mask = df[df['comment'].str.contains('|'.join(variants), case=False, na=False)]
        # Extrahiere die Kommentare als Liste
        matching_comments = mask['comment'].tolist()
        
        for comment in matching_comments:
            # Hinzufügen der Ergebnisse zur Liste
            results.append({
                'club': club_name,
                'player': player_name,
                'comment': comment
            })

    df_players = pd.DataFrame(results)

    return df_players



In [23]:
df = preprocess_json(["Alexander Nübel", "Pascal Stenzel"])

{'nübel', 'alexander nübel'}
{'pascal stenzel', 'stenzel'}


In [24]:
df

,club,player,comment
0,vfb,Alexander Nübel,Silas fehlt halt enorm. Das hat man gerade geg...
1,vfb,Alexander Nübel,"Mit Nübel im Tor leider nichts zu holen, ist l..."
2,vfb,Alexander Nübel,"Absolut starke Leistung, Respekt! 👏🏽 Leider ge..."
3,vfb,Alexander Nübel,Starke Leistung gegen die beste Mannschaft der...
4,vfb,Alexander Nübel,Fakt ist Nübel hat den Punkt verkackt
...,...,...,...
118,vfb,Pascal Stenzel,Wirkt sehr müde und Motivationslos auf mich. I...
119,vfb,Pascal Stenzel,Also wie man Stenzel Fußballgott nennen kann i...
120,vfb,Pascal Stenzel,Stenzel nicht tragbar
121,vfb,Pascal Stenzel,Wie wäre es mal mit nem gescheiten RV? Stenzel...


In [ ]:
# Liste zur Speicherung der Ergebnisse
results = []

# Sicherstellen, dass die 'comment'-Spalte keine NaN-Werte enthält
df['comment'] = df['comment'].fillna('')

# Um Leistung zu verbessern, konvertieren wir die 'comment'-Spalte einmalig zu Kleinbuchstaben
df['comment_lower'] = df['comment'].str.lower()

# Verarbeitung jedes Vereins und seiner Spieler
for club in data['clubs']:
    club_name = club['club_name']
    # Filtern der Kommentare, die zu diesem Verein gehören
    df_club = df[df['verein'] == club_name].copy()

    df_club["comment"]
    
    # Überprüfen der Spalten in df_club
    print(f"Verarbeite Verein: {club_name}")
    print("Spalten in df_club:", df_club.columns.tolist())
    
    for player_name in club['players']:
        # Bereinigung des Spielernamens
        player_name = player_name.strip()
        
        # Aufteilen des Namens in Teile
        name_parts = player_name.split()
        # Erstellen von Varianten des Namens
        variants = set()
        variants.add(player_name.lower())  # Voller Name
        # Vorname(n)
        '''
        if len(name_parts) > 1:
            first_name = ' '.join(name_parts[:-1]).lower()
            variants.add(first_name)
        else:
            first_name = name_parts[0].lower()
            variants.add(first_name)
        '''
        # Nachname(n)
        last_name = name_parts[-1].lower()
        variants.add(last_name)
        print(variants)

        mask = df_club[df_club['comment'].str.contains('|'.join(variants), case=False, na=False)]
        # Extrahiere die Kommentare als Liste
        matching_comments = mask['comment'].tolist()
        
        '''
        # Erstellen des Regex-Musters
        pattern = r'\b(' + '|'.join(re.escape(v) for v in variants) + r')\b'
        regex = re.compile(pattern)
        
        # Anwenden des Regex auf die 'comment_lower'-Spalte
        mask = df_club['comment_lower'].apply(lambda x: bool(regex.search(x)))
        
        # Überprüfen der Spalten in df_club[mask]
        print(f"Verarbeite Spieler: {player_name}")
        print("Spalten in df_club[mask]:", df_club.loc[mask].columns.tolist())
        
        # Abrufen der passenden Kommentare mit .loc
        matching_comments = df_club.loc[mask, 'comment'].tolist()
        '''
        # Hinzufügen der Ergebnisse zur Liste
        results.append({
            'club': club_name,
            'player': player_name,
            'comments': matching_comments
        })